In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

# 画像の前処理を定義
transform = transforms.Compose([
    transforms.Resize((64, 64)),  # 画像サイズの変更
    transforms.ToTensor(),  # テンソルに変換
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # 正規化
])

# データセットのロード
train_dataset = ImageFolder(root='./data/train', transform=transform)
valid_dataset = ImageFolder(root='./data/valid', transform=transform)
test_dataset = ImageFolder(root='./data/test', transform=transform)

# データローダーの設定
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)  # 入力チャネル3、出力チャネル6、カーネルサイズ5
        self.pool = nn.MaxPool2d(2, 2)  # プーリング層
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 13 * 13, 120)  # 全結合層
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 13)  # 出力層のクラス数

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# モデルのインスタンス化
net = SimpleCNN()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

for epoch in tqdm(range(10)):  # エポック数
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 100 == 99:    # 100ミニバッチごとにログを表示
            print(f'Epoch {epoch + 1}, Batch {i + 1}: Loss: {running_loss / 100:.2f}')
            running_loss = 0.0

print('Finished Training')

correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the test images: {100 * correct / total:.2f}%')


In [ ]:
# モデルを保存
PATH = './model/cnn.pth'
torch.save(net.state_dict(), PATH)


推論

In [ ]:
import torch
from torchvision import transforms
from PIL import Image
import torch.nn.functional as F

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 13 * 13, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 13)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = SimpleCNN()
net.load_state_dict(torch.load('./model/best_cnn.pth'))
net.eval()

transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

image_path = './data/test/13/S__17080326_0.jpg_22.jpg'
image = Image.open(image_path)
image = transform(image).unsqueeze(0)  # バッチ次元の追加


outputs = net(image)
_, predicted = torch.max(outputs, 1)
predicted_label = predicted.item()

print(f'Predicted Label: {predicted_label}')


In [17]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

test_dataset = ImageFolder(root='./data/test/', transform=transform)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 13 * 13, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 13)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = SimpleCNN()
net.load_state_dict(torch.load('./model/best_cnn.pth'))
net.eval()

correct = 0
total = 0
with torch.no_grad():
    for i, data in enumerate(test_loader):
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        print('Image Path:', test_dataset.samples[i][0])  # 画像のパスを出力
        print('Predicted Label:', predicted)
        print('Actual Label:', labels)
        correct += (predicted == labels).sum().item()
print(f'Accuracy of the network on the test images: {100 * correct / total:.2f}%')


Image Path: ./data/test/1/S__17080395_0.jpg_11.jpg
Predicted Label: tensor([0])
Actual Label: tensor([0])
Image Path: ./data/test/1/S__17080395_0.jpg_25.jpg
Predicted Label: tensor([0])
Actual Label: tensor([0])
Image Path: ./data/test/1/S__17080395_0.jpg_37.jpg
Predicted Label: tensor([0])
Actual Label: tensor([0])
Image Path: ./data/test/1/S__17080395_0.jpg_4.jpg
Predicted Label: tensor([0])
Actual Label: tensor([0])
Image Path: ./data/test/1/S__17080395_0.jpg_43.jpg
Predicted Label: tensor([0])
Actual Label: tensor([0])
Image Path: ./data/test/1/S__17080395_0.jpg_48.jpg
Predicted Label: tensor([0])
Actual Label: tensor([0])
Image Path: ./data/test/1/S__17080395_0.jpg_5.jpg
Predicted Label: tensor([0])
Actual Label: tensor([0])
Image Path: ./data/test/1/S__17080396_0.jpg_26.jpg
Predicted Label: tensor([0])
Actual Label: tensor([0])
Image Path: ./data/test/1/S__17080396_0.jpg_35.jpg
Predicted Label: tensor([0])
Actual Label: tensor([0])
Image Path: ./data/test/1/S__17080396_0.jpg_43.j